In [1]:
!pip install torch torchvision

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [15]:
class RobustCNN(nn.Module):
    def __init__(self):
        super(RobustCNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32,64,kernel_size = 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size= 3, padding = 1 ),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 7 * 7,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

        self._initialize_weights()

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.classifier(out)
        return out 

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [6]:

if torch.cuda.is_available:
    device = 'cuda'
else: 
    device = 'cpu'

print(device)

cuda


In [8]:
config = {
    "epochs" : 5,
    "batch_size" : 64,
    "learning_rate" : 0.001,
    "architetcure" : "RobustCNN",
    "dataset" : "FashionMNIST"
}

In [9]:
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1,0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform_train) # transform_train 적용
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform_test) # transform_test 적용

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 214kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.96MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 25.8MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [16]:
model = RobustCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

for epoch in range(config['epochs']):
  model.train()

  running_loss = 0.0
  correct_train = 0
  total_train = 0

  for i, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total_train += labels.size(0)
    correct_train += (predicted == labels).sum().item()

  train_acc = 100 * correct_train / total_train
  avg_loss = running_loss / len(train_loader)

  model.eval()

  correct_test = 0
  total_test = 0

  with torch.no_grad():
    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)

      total_test += labels.size(0)
      correct_test += (predicted == labels).sum().item()

  test_acc = 100 * correct_test / total_test

  print(f"Epoch [{epoch+1}/{config['epochs']}] "
          f"Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")


print("전체 학습 종료")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [1/5] Train Loss: 0.5344 | Train Acc: 80.22% | Test Acc: 86.15%
Epoch [2/5] Train Loss: 0.3886 | Train Acc: 85.64% | Test Acc: 89.79%
Epoch [3/5] Train Loss: 0.3497 | Train Acc: 87.20% | Test Acc: 88.55%
Epoch [4/5] Train Loss: 0.3205 | Train Acc: 88.30% | Test Acc: 90.87%
Epoch [5/5] Train Loss: 0.3048 | Train Acc: 88.86% | Test Acc: 91.09%
전체 학습 종료
